In [ ]:
#Bayesian Optimization using Gaussian Processes for Hyperparameter Tuning with noisy experimental data.

import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.models.transforms import Standardize, Normalize

%matplotlib inline
%load_ext autoreload
%autoreload 2

torch.manual_seed(0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# define ititial sample number
initial_sample = 24

# Define bounds for the parameters
bounds = torch.tensor([
    [6.0, 20.0],  # Lower bounds ph, temp
    [8.0, 40.0]   # Upper bounds ph, temp
], dtype=torch.double)

def maximin_lhs(initial_sample, bounds, n_candidates = 1000):
    if bounds is not None:
        bounds = bounds.to(torch.float64)
        d = bounds.shape[1] # double check this when adding dimensions
    else: 
        raise ValueError("Bounds must be provided to infer dimensionality")
    
    best, max_min_dist = None, -float("inf")
    for _ in range(n_candidates):
        x = (torch.rand(initial_sample, d) + torch.randperm(initial_sample)[:, None])/initial_sample
        min_dist = torch.cdist(x, x).add(1e5*torch.eye(initial_sample)).min()
        if min_dist > max_min_dist:
            best, max_min_dist = x.clone(), min_dist
    lower, upper = bounds[0], bounds[1]
    best = lower+(upper-lower)* best

    return best



def objective_function(X, pHopt =7 , temp_opt =35, a = 100, b = 1, c = 1, noise_level = 0.1):
    """
    Simulates a 2D Gaussian-like response surface with controllable noise.

    Parameters:
    - X: input tensor of shape [n, 2], columns are [pH, temp]
    - pHopt, temp_opt: optimal pH and temperature
    - a: peak value (must be high enough to keep output positive)
    - b, c: curvature coefficients (bigger = narrower peak)
    - noise_level: fraction of y to scale the noise (e.g., 0.1 = 10%)

    Returns:
    - y: simulated noisy response values
    """
    pH, temp = X[:, 0], X[:, 1]
    y = a-b*(pH-pHopt)**2 - c*(temp-temp_opt)**2
    noise = noise_level * y * torch.randn_like(y)
    return y + noise




model = SingleTaskGP(train_X, train_y, input_transform = Normalize(d=2), outcome_transform = Standardize(m=1)) # might want to set noise variance (was that through the ExactGP model?)
mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_mll(mll)




In [ ]:
x_init = maximin_lhs(initial_sample, bounds)
print(x_init)